# Question 5

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score, make_scorer
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import neighbors, datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import scikitplot as skplt
from sklearn.naive_bayes import MultinomialNB

import warnings
warnings.filterwarnings("ignore", category = UserWarning)

In [2]:
df = pd.read_csv('/Users/lakshyagarg/Desktop/Fall_sem/Predictive_analytics/HW2_grid_search/car.csv')

# Visualize data

In [3]:
print (df.head(5))
print (df['buying'].unique())
print (df['maint'].unique())
print (df['lug_boot'].unique())
print (df['safety'].unique())
print (df['Class'].unique())

print (df.shape)
df.info()
print(df.isna().sum())

print (df.columns)




  buying  maint doors persons lug_boot safety  Class
0  vhigh  vhigh     2       2    small    low  unacc
1  vhigh  vhigh     2       2    small    med  unacc
2  vhigh  vhigh     2       2    small   high  unacc
3  vhigh  vhigh     2       2      med    low  unacc
4  vhigh  vhigh     2       2      med    med  unacc
['vhigh' 'high' 'med' 'low']
['vhigh' 'high' 'med' 'low']
['small' 'med' 'big']
['low' 'med' 'high']
['unacc' 'acc' 'vgood' 'good']
(1728, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   Class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB
buying      0
maint       0
doors       0
persons  

In [4]:
df_g = df.groupby(['Class']).size()
print (df_g)

Class
acc       384
good       69
unacc    1210
vgood      65
dtype: int64


# Converting in one hot encoding

In [5]:
y_buy = pd.get_dummies(df.buying, prefix='buying')
y_maint = pd.get_dummies(df.maint, prefix='maint')
y_lug_boot = pd.get_dummies(df.lug_boot, prefix='lug_boot')
y_safety = pd.get_dummies(df.safety, prefix='safety')


df = df.drop(['buying','maint','lug_boot','safety'], axis = 'columns')
df = pd.concat([df,y_buy,y_maint,y_lug_boot,y_safety], axis='columns')

df['persons'] = df['persons'].apply(lambda x: 5 if x == 'more' else int(x))
df['doors'] = df['doors'].apply(lambda x: 5 if x == '5more' else int(x))

def Class(x):
    if x == 'vgood':
        return 4
    elif x == 'good':
        return 3
    elif x == 'acc':
        return 2
    else:
        return 1
    
df['Class'] = df['Class'].apply(Class)

print(df.columns)

In [6]:
df.head(5)

,buying,maint,doors,persons,lug_boot,safety,Class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


# Converting in label encoding

In [7]:
# def buying(x):
#     if x == 'vhigh':
#         return 4
#     elif x == 'high':
#         return 3
#     elif x == 'med':
#         return 2
#     else:
#         return 1
    
    
# def maint(x):
#     if x == 'vhigh':
#         return 4
#     elif x == 'high':
#         return 3
#     elif x == 'med':
#         return 2
#     else:
#         return 1
    

# def lugboot(x):
#     if x == 'big':
#         return 3
#     elif x == 'med':
#         return 2
#     else:
#         return 1


# def safety(x):
#     if x == 'high':
#         return 3
#     elif x == 'med':
#         return 2
#     else:
#         return 1
    
    
# def Class(x):
#     if x == 'vgood':
#         return 4
#     elif x == 'good':
#         return 3
#     elif x == 'acc':
#         return 2
#     else:
#         return 1


# df['buying'] = df['buying'].apply(buying)

# df['maint'] = df['maint'].apply(maint)

# df['lug_boot'] = df['lug_boot'].apply(lugboot)

# df['safety'] = df['safety'].apply(safety)

# df['Class'] = df['Class'].apply(Class)

# df['persons'] = df['persons'].apply(lambda x: 5 if x == 'more' else int(x))

# df['doors'] = df['doors'].apply(lambda x: 5 if x == '5more' else int(x))

In [8]:
df.head(5)
df.describe

<bound method NDFrame.describe of       buying  maint  doors  persons  lug_boot  safety  Class
0          4      4      2        2         1       1      1
1          4      4      2        2         1       2      1
2          4      4      2        2         1       3      1
3          4      4      2        2         2       1      1
4          4      4      2        2         2       2      1
...      ...    ...    ...      ...       ...     ...    ...
1723       1      1      5        5         2       2      3
1724       1      1      5        5         2       3      4
1725       1      1      5        5         3       1      1
1726       1      1      5        5         3       2      3
1727       1      1      5        5         3       3      4

[1728 rows x 7 columns]>

# Train test split

In [9]:
df_train = df.drop(['Class'], axis = 'columns')
df_label = df['Class']
X_train, X_test, y_train, y_test = train_test_split(df_train, df_label, test_size=0.20, random_state=42)

In [10]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(1382, 6)
(1382,)
(346, 6)
(346,)


In [11]:
df.head(5)

,buying,maint,doors,persons,lug_boot,safety,Class
0,4,4,2,2,1,1,1
1,4,4,2,2,1,2,1
2,4,4,2,2,1,3,1
3,4,4,2,2,2,1,1
4,4,4,2,2,2,2,1


In [12]:
# Nested CV

In [13]:
inner_cv = KFold(n_splits = 4, shuffle = False )
outer_cv = KFold(n_splits =4, shuffle = False )
scoring = {'acc':'accuracy','balanced_accuracy':'balanced_accuracy', 'recall_micro':'recall_micro'}



In [14]:
decisiontree =  DecisionTreeClassifier(random_state = 42)
logistic_reg =  LogisticRegression()
knn = KNeighborsClassifier()
SVM = SVC(probability=True)
naive_bayes= MultinomialNB()

In [15]:
# Decision tree grid
max_depth_options = list(range(1,20))
min_samples_leaf_options = list (range(1,6))
dt_grid = dict(max_depth = max_depth_options, min_samples_leaf = min_samples_leaf_options)

# Logistic regression grid
max_iter_options = [1000]
penalty_options = ["l1", "l2"]
solver_options = ['saga']
C_options = [1,100,500,1000]
lr_grid = dict(penalty = penalty_options, C = C_options, max_iter = max_iter_options, solver = solver_options)

# KNN grid
k_range_options = list(range(1,16))
weight_options = ["uniform", "distance"]
knn_grid = dict(n_neighbors = k_range_options, weights = weight_options)

# SVM grid
svm_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [16]:
dt_clf = GridSearchCV(estimator=decisiontree, param_grid=dt_grid, scoring = scoring, cv=inner_cv,refit='acc')
lr_clf = GridSearchCV(estimator=logistic_reg, param_grid=lr_grid, scoring = scoring, cv=inner_cv,refit='acc')
knn_clf = GridSearchCV(estimator=knn, param_grid=knn_grid, scoring = scoring, cv=inner_cv,refit='acc')
svr_clf = GridSearchCV(estimator=SVM, param_grid=svm_grid, scoring = scoring, cv=inner_cv,refit='acc')



In [17]:
# Nested CV with parameter optimization
dt_score = cross_validate(dt_clf, X= X_train, y=y_train, cv=outer_cv, scoring = scoring)
# dt_scores = dt_score.mean()
lr_score = cross_validate(lr_clf, X=X_train, y=y_train, cv=outer_cv, scoring = scoring)
# lr_scores = lr_score.mean()
knn_score = cross_validate(knn_clf, X=X_train, y=y_train, cv=outer_cv, scoring = scoring)
# knn_scores = knn_score.mean()
svr_score = cross_validate(svr_clf, X=X_train, y=y_train, cv=outer_cv, scoring = scoring)
# svr_scores = svr_score.mean()
naive_bayes_score = cross_validate(naive_bayes, X=X_train, y=y_train , cv=outer_cv, scoring = scoring)
# naive_b_scores = naive_bayes_score.mean()
print (naive_bayes_score)

{'fit_time': array([0.00126219, 0.00098205, 0.00097108, 0.00098181]), 'score_time': array([0.00141788, 0.00134015, 0.00133586, 0.00134683]), 'test_acc': array([0.70809249, 0.74566474, 0.71014493, 0.65797101]), 'test_balanced_accuracy': array([0.25, 0.25, 0.25, 0.25]), 'test_recall_micro': array([0.70809249, 0.74566474, 0.71014493, 0.65797101])}


In [18]:
# y_pred_test_nb =naive_bayes.predict(X_test)
# print (classification_report(y_test, y_pred_test_nb))

In [19]:
dt_score =pd.DataFrame(dt_score)
dt_score['model'] = 'Decision Tree'

lr_score = pd.DataFrame(lr_score)
lr_score['model'] = 'Logistic Regression'

knn_score =  pd.DataFrame(knn_score)
knn_score['model'] = 'K nearest neighbour'

svr_score = pd.DataFrame(svr_score)
svr_score['model'] = 'Support Vector'

naive_bayes_score = pd.DataFrame(naive_bayes_score)
naive_bayes_score['model'] = 'Naive Bayes'

df_all_scores = pd.concat([dt_score,lr_score,knn_score,svr_score,naive_bayes_score], axis = 'rows')


df_all_scores = df_all_scores.astype({'model': 'string'})

# print (df_all_scores.dtypes)
df_all_scores = df_all_scores[['test_acc','test_balanced_accuracy','test_recall_micro','model']]
df_all_scores = df_all_scores.groupby(['model']).mean()
df_all_scores = df_all_scores.reset_index()
print (df_all_scores)
df_all_scores.to_csv('/Users/lakshyagarg/Downloads/scores.csv')

                 model  test_acc  test_balanced_accuracy  test_recall_micro
0        Decision Tree  0.975392                0.939558           0.975392
1  K nearest neighbour  0.943537                0.836284           0.943537
2  Logistic Regression  0.853093                0.723971           0.853093
3          Naive Bayes  0.705468                0.250000           0.705468
4       Support Vector  0.978292                0.958867           0.978292


In [20]:
# Based on the accuracy score SVM is the best model, when we do one hot encoding
# Retraining the SVM model we get
svr_clf_2 = GridSearchCV(SVM, svm_grid, cv = inner_cv, scoring = scoring,refit='acc')
svr_clf_2.fit(X_train,y_train)

y_pred_test = svr_clf_2.predict(X_test)

print (classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       235
           2       0.97      0.93      0.95        83
           3       0.59      0.91      0.71        11
           4       1.00      0.82      0.90        17

    accuracy                           0.97       346
   macro avg       0.89      0.92      0.89       346
weighted avg       0.98      0.97      0.97       346

